In [2]:
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
import pickle

In [3]:
def predict_sequence_3(infenc, infdec, source, n_steps):
    # encode
    state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3 = infenc.predict(source)  # source_dim = ()
#decoder_input = source[:,-1,:,:,:].reshape((1,1,64,64,1))
    decoder_input = np.repeat(0,128*144).reshape((1,1,128,144,1))
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h_1, c_1, h_2, c_2, h_3, c_3 = infdec.predict([decoder_input, state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3 = h_1, c_1, h_2, c_2, h_3, c_3
        # update target sequence
        decoder_input = yhat
    return np.array(output)

In [7]:
def prediccion(encoder, decoder, steps):
    #y_pred = np.zeros((251, 4, 128, 144,1)) ## para 4
    y_pred = np.zeros((125, 8, 128, 144,1)) ## para 8

    for i in range(y_test.shape[0]):
        pred3 = predict_sequence_3(encoder, decoder, x_test[np.newaxis,i,:,:,:,:], steps)
        y_pred[i] = pred3
    return y_pred

### Carga de datos

In [4]:
Y_train = np.load("/opt/datos/dataset/gfs/Y.npy")
print(Y_train.shape)

(19288, 137, 157)


In [5]:
### t - 4

train = Y_train.reshape(2411,8,137,157,1)
Y_train = None
print(train.shape)

x_test  = train[2160:, 0:4, 0:128, 0:144, :]
y_test  = train[2160:, 4:8, 0:128, 0:144, :]

print(x_test.shape)
print(y_test.shape)

(2411, 8, 137, 157, 1)
(251, 4, 128, 144, 1)
(251, 4, 128, 144, 1)


### 01: LSTM - 10 EPOCH - Adam - tiempos: (t - 4/t + 4) - loss: mse 

In [6]:
model = load_model('/opt/datos/entrenamientos/gfs/lstm/01/model.h5')
encoder = load_model('/opt/datos/entrenamientos/gfs/lstm/01/encoder.h5')
decoder = load_model('/opt/datos/entrenamientos/gfs/lstm/01/decoder.h5')

y_pred = prediccion(encoder, decoder, 4)

(251, 4, 128, 144, 1)


### Error de la predicción a 3 hs.

In [7]:
y_pred2 = np.zeros((251, 128, 144,1))

y_test2 = np.zeros((251, 128, 144,1))

for i in range(y_test.shape[0]):
    y_pred2[i] = y_pred[i,0,:,:,:]
    y_test2[i] = y_test[i,0,:,:,:]

In [8]:
y_test2 = y_test2.flatten()
y_pred2 = y_pred2.flatten()

##MEAN SQUARE ERROR

mae = MAE(y_test2, y_pred2)
print(f"MAE: {mae}")

(4626432,)
(4626432,)
MAE: 0.30991631865097324


### Para todas las predicciones

In [9]:
y_test_f = y_test.flatten()
print(y_test.shape)
y_pred_f = y_pred.flatten()
print(y_pred.shape)

(251, 4, 128, 144, 1)
(251, 4, 128, 144, 1)


In [10]:
## MEAN ABSOLUTE ERROR

mae = MAE(y_test_f, y_pred_f)
print(f"MAE: {mae}")

##MEAN SQUARE ERROR

mse = MSE(y_test_f, y_pred_f)
print(f"MSE: {mse}")

## root mean square error

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

MAE: 0.4076995157944005
MSE: 1.7127807239754358
RMSE: 1.308732487552531


### 03: LSTM - 10 EPOCH - SGD (lr=0.01 / decay=1e-6 / momentum=0.9 / nesterov=True) - 
###     tiempos: (t - 4/t + 4) - loss: mae 

In [11]:
model = load_model('/opt/datos/entrenamientos/gfs/lstm/03/model.h5')
encoder = load_model('/opt/datos/entrenamientos/gfs/lstm/03/encoder.h5')
decoder = load_model('/opt/datos/entrenamientos/gfs/lstm/03/decoder.h5')

y_pred = prediccion(encoder, decoder, 4)

(251, 4, 128, 144, 1)


### prediccion a 3 hs

In [12]:
y_pred2 = np.zeros((251, 128, 144,1))
y_test2 = np.zeros((251, 128, 144,1))

for i in range(y_test.shape[0]):
    y_pred2[i] = y_pred[i,0,:,:,:]
    y_test2[i] = y_test[i,0,:,:,:]

In [13]:
y_test2 = y_test2.flatten()
print(y_test2.shape)
y_pred2 = y_pred2.flatten()
print(y_pred2.shape)

##MEAN SQUARE ERROR

mae = MAE(y_test2, y_pred2)
print(f"MAE: {mae}")

(4626432,)
(4626432,)
MAE: 0.27537576963794236


### Todas las predicciones

In [14]:
y_test_f = y_test.flatten()
print(y_test.shape)
y_pred_f = y_pred.flatten()
print(y_pred.shape)

## MEAN ABSOLUTE ERROR

mae = MAE(y_test_f, y_pred_f)
print(f"MAE: {mae}")

##MEAN SQUARE ERROR

mse = MSE(y_test_f, y_pred_f)
print(f"MSE: {mse}")

## root mean square error

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

(251, 4, 128, 144, 1)
(251, 4, 128, 144, 1)
MAE: 0.3545433206699246
MSE: 2.0316725633052983
RMSE: 1.4253675186790593


### 04: LSTM - 10 EPOCH - Adam - tiempos: (t - 8/t + 8) - loss: mse 

In [5]:
Y_train = np.load("/opt/datos/dataset/gfs/Y.npy")
print(Y_train.shape)

new_train = Y_train[:19280,:,:]
train = new_train.reshape(1205,16,137,157,1)
Y_train = None
new_train = None
print(train.shape)

x_test  = train[1080:, 0:8, 0:128, 0:144, :]
y_test  = train[1080:, 8:16, 0:128, 0:144, :]

print(x_test.shape)
print(y_test.shape)

(19288, 137, 157)
(1205, 16, 137, 157, 1)
(125, 8, 128, 144, 1)
(125, 8, 128, 144, 1)


In [8]:
model = load_model('/opt/datos/entrenamientos/gfs/lstm/04/model.h5')
encoder = load_model('/opt/datos/entrenamientos/gfs/lstm/04/encoder.h5')
decoder = load_model('/opt/datos/entrenamientos/gfs/lstm/04/decoder.h5')

y_pred = prediccion(encoder, decoder, 8)

### prediccion a 3 hs

In [9]:
y_pred2 = np.zeros((251, 128, 144,1))
y_test2 = np.zeros((251, 128, 144,1))

for i in range(y_test.shape[0]):
    y_pred2[i] = y_pred[i,0,:,:,:]
    y_test2[i] = y_test[i,0,:,:,:]
    
y_test2 = y_test2.flatten()
print(y_test2.shape)
y_pred2 = y_pred2.flatten()
print(y_pred2.shape)

##MEAN SQUARE ERROR

mae = MAE(y_test2, y_pred2)
print(f"MAE: {mae}")

(4626432,)
(4626432,)
MAE: 0.1892250633088179


### Todas las predicciones:

In [10]:
y_test_f = y_test.flatten()
print(y_test.shape)
y_pred_f = y_pred.flatten()
print(y_pred.shape)

## MEAN ABSOLUTE ERROR

mae = MAE(y_test_f, y_pred_f)
print(f"MAE: {mae}")

##MEAN SQUARE ERROR

mse = MSE(y_test_f, y_pred_f)
print(f"MSE: {mse}")

## root mean square error

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

(125, 8, 128, 144, 1)
(125, 8, 128, 144, 1)
MAE: 0.9454349728817787
MSE: 2.8952452300193205
RMSE: 1.7015420153552838
